In [ ]:
import numpy as np
import pandas as pd
from plotly import express as px

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv('reg_data/train_data.csv', index_col=0, parse_dates=['date'])
df.reset_index(drop=True, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.date[34]

In [ ]:
# .sort_values(['tic','date'])[['tic','date',]
wat = df.groupby("tic", as_index=True)['close'].ewm(com=60, adjust=True).std()
wat = wat.droplevel(0)

In [ ]:
wat.index

In [ ]:
wat

In [ ]:
df["volatility"] = wat

In [ ]:
df

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "volatility", color = 'tic',  )

In [ ]:
px.line(df[df.tic == "BA"], x = 'date', y = "close", color = 'tic',  )

In [ ]:
delta = 0.9836065
delta/(1-delta)
I = 1000

w = [(1-delta) * (delta**i) * i for i in range(I)]

In [ ]:

data = {'r_t': np.random.randint(0, 100, I)} # Example returns between -0.5 and 0.5
df = pd.DataFrame(data)

df['gains'] = df['r_t'].cumsum()
df

In [ ]:
df['r_t_hat'] = df['r_t'].ewm(com=60, adjust=False).mean() 

In [ ]:
df['std_r_t'] = df['r_t'].ewm(com=60, adjust=True).std() 

In [ ]:
df['sig'] = np.sqrt( df['std_r_t'] * 261)

In [ ]:
df

In [ ]:
df["r_t"][:25]

In [ ]:
I = 1000

t = 23

def sig_sq(t, delta, df):
    r = df['r_t']
    rh = df['r_t_hat']
    sc = 261 # Scaling constant, assuming 261 trading days in a year
    return sc * sum([(1-delta) * (delta**i) * (r[t-1-i] - rh[t])**2 for i in range(t)])


df["volat"] = [ np.sqrt(sig_sq(t, delta, df)) for t in range(0, I) ]


In [ ]:
fig = px.line(df, y="volat")
fig#.add_trace(px.line(df, y="r_t").data[0])

In [ ]:
fig = px.scatter(df, y="sig", x = "volat")
fig

In [ ]:
import pandas as pd
import numpy as np

class bla:

    def __init__(self):
        self.df = pd.read_csv("sample_df.csv", index_col='day')
        self.day = 345

    def calc_periods_returns(self, period, days):
        #data =  self.df[self.df.day == self.day] #, ['volatility', 'tic']]
        data = self.df.loc[self.day, ['date_idx', 'volatility', 'tic']].set_index('tic')

        data['ret'] = self.df.loc[self.day - days : self.day, ['ret','tic']].groupby('tic').sum()['ret']
        data['returns'] = data['ret'] / (data['volatility'] * np.sqrt(days))
        data.set_index('date_idx', inplace = True)
        self.df.loc[self.day, period] = data['returns'] 
        #print(self.df.loc[self.day])



    def doit(self, rng = [5]):
        periods = {'ret_1M': 21,
            'ret_2M': 42,
            'ret_3M': 63,
            'ret_1Y': 252} 
        for i in rng:
            self.day = i
            for p,d in periods.items():
                self.calc_periods_returns(p,d)    

In [ ]:
import pandas as pd
import numpy as np

class bla:

    def __init__(self):
        self.df = pd.read_csv("sample_df.csv").set_index(['day','tic'], drop=False)

        self.df = self.df[['day', 'tic', 'close', 'macd', 'rsi_30', 'volatility',
       'ret', 'ret_1M', 'ret_2M', 'ret_3M', 'ret_1Y']]

        self.df.fillna({'volatility':1}, inplace=True)

        self.periods = ['ret_1M', 'ret_2M', 'ret_3M', 'ret_1Y']
        self.days = [21,42,63,252]
        self.time_scale = pd.DataFrame(np.sqrt([self.days]), columns = self.periods)

        #self.periods = pd.DataFrame([[21,42,63,252]], columns= ['ret_1M', 'ret_2M', 'ret_3M', 'ret_1Y'])
        self.df[self.periods] = 0.0

    def calc_periods_returns(self):
        volatility = self.df.loc[(self.day, slice(None)), ['volatility']]
        volatility.columns = pd.RangeIndex(start=0, stop=1, step=1)
        
        norm = 1 / (volatility @ self.time_scale)
               
               
        addition = self.df.loc[self.day - 1,['ret','ret','ret','ret']] 
        addition.columns = self.periods
        

        s = np.maximum(self.day - np.array(self.days),np.array([0]))
        tmp = {k:v for k,v in zip(s,self.periods)}
        lower = self.df.loc[(s, slice(None)),'ret']
        lower = lower.unstack(level=0, )
        lower.columns = lower.columns.map(tmp)

        addition -= lower

        addition.fillna(0.0, inplace= True)

        self.tracking += addition

        update = self.tracking * norm
        #update.index = update.index.set_levels([self.day -1],level=0)

        #self.df[self.periods].update(update)
        #self.df.loc[(self.day,slice(None)), periods] = self.tracking.loc[:,periods]  / (volatility * np.sqrt(days))
        self.df.loc[self.day,self.periods] = update


    def doit(self, rng = [5]):
        self.dim = self.df.tic.nunique()
        
        # self.tracking = {'ret_1M': np.array([0.0]*self.dim ),
        #         'ret_2M':  np.array([0.0]*self.dim ),
        #         'ret_3M':  np.array([0.0]*self.dim ),
        #         'ret_1Y':  np.array([0.0]*self.dim )} 
        
        self.tracking = self.df.loc[0,self.periods ]
        l = len(rng)
        for i in rng:
            print('\r line', i, " (", int(i/l)*100 , " %)" , end='')
            self.day = i
            #for p,d in self.periods.items():
            self.calc_periods_returns()


In [ ]:
B = bla()
B.df

In [ ]:
B.df.columns

In [ ]:
what = B.df.index.get_level_values('day').unique()[1:]
what

In [ ]:

B.doit(what)
#B.doit(list(range(67,100)))

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f bla.calc_periods_returns B.doit(what)

In [ ]:
B.df

In [ ]:
B.df.index

In [ ]:
lal = B.df.xs('ZT=F',level='tic')

In [ ]:
lal

In [ ]:
from plotly import express as px
px.line(lal, y = 'ret_1M')

In [ ]:
a =  list(range(30,10,-1))
a

In [ ]:
sm = 0

w = 6
for e in range(1,20): 
    s = max(0,e-w)

    sm -= a[s]
    sm += a[e-1]
    bad = sum(a[s+1:e])
    print(e, s, e, a[s+1:e], bad, sm)

In [ ]:
sm = 0

w = 6
for e in range(1,len(a)): 
    s = e-w 
    if (s >= 0):
        sm -= a[s]
    
    sm += a[e-1]
    bad = a[max(0,s+1):e]
    print(e, s, e, bad, sum(bad), sm)

In [ ]:

def step(a, e, w, sm):
    s = e-w 
    if (s >= 0):
        sm -= a[s]
    
    sm += a[e-1]
    bad = a[max(0,s+1):e]
    print(e, s, e, bad, sum(bad), sm)
    return sm


sm = 0

w = 6
for e in range(1,len(a)): 
    sm = step(a, e, w, sm)

In [ ]:
sum([24, 23, 22, 21, 20, 19]) 

In [ ]:
99-85